# PyUL2 Parameter Scanner (For Linear BH Motion)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PyUL2-Parameter-Scanner-(For-Linear-BH-Motion)" data-toc-modified-id="PyUL2-Parameter-Scanner-(For-Linear-BH-Motion)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PyUL2 Parameter Scanner (For Linear BH Motion)</a></span></li><li><span><a href="#Initialization" data-toc-modified-id="Initialization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Initialization</a></span><ul class="toc-item"><li><span><a href="#Graphics" data-toc-modified-id="Graphics-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Graphics</a></span></li></ul></li><li><span><a href="#Getting-Ready" data-toc-modified-id="Getting-Ready-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Getting Ready</a></span><ul class="toc-item"><li><span><a href="#Setting-up-the-scan" data-toc-modified-id="Setting-up-the-scan-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Setting up the scan</a></span></li></ul></li><li><span><a href="#Batch-Execution" data-toc-modified-id="Batch-Execution-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Batch Execution</a></span></li><li><span><a href="#Batch-Processing." data-toc-modified-id="Batch-Processing.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Batch Processing.</a></span><ul class="toc-item"><li><span><a href="#Orbital-Radius-Decay-Diagram" data-toc-modified-id="Orbital-Radius-Decay-Diagram-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Orbital Radius Decay Diagram</a></span></li><li><span><a href="#Lowest-Energy-Achieved" data-toc-modified-id="Lowest-Energy-Achieved-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Lowest Energy Achieved</a></span></li><li><span><a href="#Loading-Single" data-toc-modified-id="Loading-Single-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Loading Single</a></span></li><li><span><a href="#2D-Preview" data-toc-modified-id="2D-Preview-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>2D Preview</a></span></li></ul></li></ul></div>

# Initialization

In [3]:
###### Do not touch
MinVersion = 23

import PyUltraLight2 as PyUL

if (PyUL.S_version < MinVersion):
    raise RuntimeError("You need the latest PyULN!")

import numpy as np

np.set_printoptions(suppress=True)

import math

import numba
import numexpr as ne
import time
import pyfftw
import os
import sys
import multiprocessing
import numpy

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec
import matplotlib.animation
from matplotlib.offsetbox import AnchoredText

import IPython
from IPython.core.display import clear_output, display, Video

from numpy import sqrt, exp, log, log10, sin, cos, tan, pi

%reload_ext autoreload
%autoreload 2

# Useful Aux Functions
ToCode = PyUL.convert
ToPhys = PyUL.convert_back
CB = PyUL.convert_between
printU = PyUL.printU

m22 = PyUL.axion_E / 1e-22

printU(f"m22 = {m22:.3g}", 'universe')

Axion Mass (eV).1e-21
PyUL.Universe: Axion Mass: 1e-21 eV.
PyUL.23.5: (c) 2020 - 2021 Wang., Y. and collaborators. 
Auckland Cosmology Group

Original PyUltraLight Team:
Edwards, F., Kendall, E., Hotchkiss, S. & Easther, R.
arxiv.org/abs/1807.04037
PyUL.universe: m22 = 10


## Graphics

In [4]:
plt.style.use('default')

plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams["text.usetex"] = False
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.facecolor'] = 'w'

from matplotlib.colors import LinearSegmentedColormap
Palette = 'magma'
cyccol = ['#000000', '#ffffff', '#000000']
divcol = ['#003262', '#005b96', '#ffffff', '#d5a756', '#B51700']
divcols = ['#005b96', '#ffffff', '#d5a756']

CycPalette = LinearSegmentedColormap.from_list('myCyc', cyccol)

DivPalette = LinearSegmentedColormap.from_list('myDiv', divcol)
DivPaletteS = LinearSegmentedColormap.from_list('myDiv', divcols)

EFigSize = (10, 12)
EFigSizeMono = (10, 6)
VideoSize = (8, 8)
FPS = 10
DPI = 72

FrameCap = 60  # Maximum Number of Frames to Animate

########################################

# BH Trajectory

Colors = np.array([[209, 17, 65], [0, 177, 89], [0, 174, 219], [243, 119, 53],
                   [255, 196, 37]]) / 255

# NBody
ENStyle = '-'
ENColor = '#ff6c0e'  # NBody

EKColor = '#8dd9e7'
EPColor = '#fd8dc8'

# ULDM
EUStyle = '-'
EUColor = '#7f1717'  # ULDM

EQColor = '#ccb603'  # Kinetic
ERColor = '#6cc25c'  # Self-Interaction
EGColor = '#2c8899'  # Particle Potential

# Totals
ETStyle = '-'
ETColor = '#080808'

# Auxiliary
HelperStyle = 'k.'
EmphColor = 'black'
GeomColor = 'olivedrab'

# Getting Ready

## Setting up the scan

*Supported scan parameters:*

    Density Resolution TM_M TM_v U_v Step_Factor Scaling Plummer_Radius
    
(Scaling refers scaling the box size by a factor.)

In [5]:
save_path = 'Batches/Test1'
PyUL.DSManagement(save_path, Force=False)

[ Batches/Test1 ] : The current size of the folder is 0.0 Mib
[ Batches/Test1 ] : Save Folder Created.


In [6]:
ScanParams = ['Density', 'TM_M', 'TM_v']

ValuePool = [[1e3,1e5,1e7],[0.1,1,10],[12.5,25,50]] # Inherit the defined units in your configuration file

In [8]:
N, SaveString = PyUL.ParameterScanGenerator('Reusable/Configurations/QDrag1_QW',
                            ScanParams,
                            ValuePool,
                            save_path,
                            SaveSpace=False,
                            KeepResol=True,
                            KeepSmooth=False,
                            AdaptiveTime=False)

PyUL.ParamScan: Automated scan will be performed over 3 parameters.
There will be 27 separate simulations. They are:
[('Density', [1000.0, 100000.0, 10000000.0]), ('TM_M', [0.1, 1, 10]), ('TM_v', [12.5, 25, 50])]
(Units are defined in the donor config file)
Generated config file for PScan_D01_M01_V01
Generated config file for PScan_D01_M01_V02
Generated config file for PScan_D01_M01_V03
Generated config file for PScan_D01_M02_V01
Generated config file for PScan_D01_M02_V02
Generated config file for PScan_D01_M02_V03
Generated config file for PScan_D01_M03_V01
Generated config file for PScan_D01_M03_V02
Generated config file for PScan_D01_M03_V03
Generated config file for PScan_D02_M01_V01
Generated config file for PScan_D02_M01_V02
Generated config file for PScan_D02_M01_V03
Generated config file for PScan_D02_M02_V01
Generated config file for PScan_D02_M02_V02
Generated config file for PScan_D02_M02_V03
Generated config file for PScan_D02_M03_V01
Generated config file for PScan_D02_M0

[**Back to the Top**](#root)
<a id='run'></a>
# Batch Execution

This creates a timestamped folder using the current settings, and the integrator stores all requested files in it too.

In [9]:
FullSim = True
T0 = time.time()
runs = os.listdir(save_path)
runs.sort()

NRuns = len(runs)

Run = 0

for run_folder in runs:

    if os.path.isdir(f'{save_path}/{run_folder}'):

        RunText = f'Simulation #{Run+1} out of {NRuns}'
        PyUL.evolve(save_path,
                    run_folder,
                    NBodyInterp=FullSim,
                    SelfGravity=FullSim,
                    NBodyGravity=True,
                    Silent=True,
                    Shift=False,
                    AutoStop=False,
                    AutoStop2=False,
                    Message=RunText,
                    Stream=False,
                    StreamChar=[1, 4])

        Run += 1

        print(f'Simulation {Run} finished.')
        time.sleep(2)

import IPython
IPython.display.Audio("Done.mp3", autoplay=True)

print(f"{time.time()-T0:.3g}s")

Build 2021 Aug 25
Message: Simulation #27 out of 28
PyUL.Runtime: Simulation name is ./Batches/Test1/PScan_D03_M03_V03
PyUL.Runtime: 128 Resolution for 32Myr
PyUL.Runtime: Simulation Started at 27/09/2021, 12:53:21.
PyUL.Runtime: Taking 8 ULDM steps


[●●●●●●●●●●●●●●●●●●●●] 100%      Exp. Time: 27/09/2021, 12:53:22 (Prev.: 0.17s) 

PyUL.Runtime: Run Complete. Time Elapsed (d:h:m:s): 0:0:0:1.38
Simulation 27 finished.
102s


NameError: name 'batch_path' is not defined

# Batch Processing.

In [3]:
### Initializing Journaling Variables

#### Dummies and Aux
TimeLog = [] # Time Series
ResoLog = [] # Resolutions
PlRaLog = [] # Plummer Radii

#### Core Variables
IniRLog = [] # Initial Radius Handle
MaRaLog = [] # Mass Ratio Handle

TMMaLog = [] # NBody Mass

DyTSLog = [] # Dynamical Timescale (Corrected by C)
IniDLog = [] # Initial Density
MEncLog = []

TMAcLog = [] # Nbody Acceleration
TMDSLog = [] # Nbody Traj
TMDVLog = [] # Nbody Velocity

# Energies

TMEKLog = [] # BH Kinetic
TMEPLog = [] # BH Potential (Due to ULDM)

ULEKLog = [] # ULDM KQ (Kinetic and Quantum)
ULEPLog = [] # ULDM Self-Grav

SysELog = [] # System Total Energy

Run = 0


In [4]:
save_path = './Batches/' + PyUL.Runs('./Batches', Automatic = False)

runs = os.listdir(save_path)
runs.sort()

Time = time.time()

for run_folder in runs:
    loc = './' + save_path + '/' + run_folder
    if os.path.isdir(loc):
        
        # Reload Config.. Why don't I use a class?
        NS, length, length_units, resol, duration, duration_units, step_factor, save_number, save_options, save_format, s_mass_unit, s_position_unit, s_velocity_unit, solitons,start_time, m_mass_unit, m_position_unit, m_velocity_unit, particles, embeds, Uniform,Density, density_unit ,a, UVel = PyUL.LoadConfig(loc)

        if save_number == -1:
            sim_number = PyUL.ULDStepEst(duration,duration_units,
                                                  length,length_units,
                                                  resol,step_factor, 
                                                  save_number = -1)
        else:
            sim_number = save_number

        # Dict of Run Data
        EndNum, Loaded = PyUL.Load_npys(loc,save_options)

        TMdata = Loaded['NBody']
        
        AccData = Loaded['DF']
        
        
        Loaded = {}

        NBo = len(particles)

        # Time
        durationMyr = CB(duration,duration_units,'Myr','t')

        # Space
        lengthKpc = CB(length,length_units,'kpc','l')
        lengthC = ToCode(length,length_units,'l')

        # Mass

        MassList_MSol = []
        MassListSI = []
        for TM in particles:
            MassList_MSol.append(CB(TM[0],m_mass_unit,'M_solar_masses','m'))
            MassListSI.append(CB(TM[0],m_mass_unit,'kg','m'))

        # Density
        DensitySI = CB(Density,density_unit,'kg/m3','d')
        DensityC = ToCode(Density,density_unit,'d')

        ## Misc. Pre-Multipliers


        # NBody_State Vector

        XPre = ToPhys(1,'kpc','l')
        VPre = ToPhys(1,'km/s','v')

        XPreSI = ToPhys(1,'m','l')
        VPreSI = ToPhys(1,'m/s','v')

        IArray = np.arange(len(TMdata[0]))

        # Dimensionful NBody State

        TMDataS = np.array(TMdata)
        TMDataSI = np.array(TMdata)

        TMDataS[:,IArray % 6 <= 2] *= XPre
        TMDataS[:,IArray % 6 >= 3] *= VPre


        TMDataSI[:,IArray % 6 <= 2] *= XPreSI
        TMDataSI[:,IArray % 6 >= 3] *= VPreSI


        # Rho data
        DPre = CB(1,density_unit,'kg/m3','d')
        # Energy
        EPre = PyUL.energy_unit
        # Field Strength
        PPre = PyUL.mass_unit / PyUL.length_unit
        
        
        # Acceleration
        APre = PyUL.length_unit/PyUL.time_unit**2
        
        AccDataSI = np.array(AccData) * APre
        AccDataSI[0,:] = AccDataSI[1,:] # not a good fix but doesn't matter
        # U ENERGY
        
        # Total Energy from Integrator
        egylist = np.load('{}{}'.format(loc, '/Outputs/egylist.npy'),allow_pickle=True) * EPre

        # Energy Due to N Body Potential
        egpcmlist =  np.load('{}{}'.format(loc, '/Outputs/egpcmMlist.npy'),allow_pickle=True) * EPre # NEW
        egpcmlist2 = np.load('{}{}'.format(loc, '/Outputs/egpcmlist.npy'),allow_pickle=True) * EPre # OLD

        # Energy Due to ULDM Self-Interaction
        egpsilist = np.load('{}{}'.format(loc, '/Outputs/egpsilist.npy'),allow_pickle=True) * EPre

        # Energy Due to Quantum Fun
        ekandqlist = np.load('{}{}'.format(loc, '/Outputs/ekandqlist.npy'),allow_pickle=True) * EPre

        mtotlist = np.load('{}{}'.format(loc, '/Outputs/masseslist.npy'),allow_pickle=True) * EPre


        # Translating Snapshots back to Time
        
        timerange = np.linspace(0,durationMyr,save_number+1)

        Tp = np.arange(EndNum)

        Tp = Tp * durationMyr / (sim_number+1)

        TpSec = PyUL.convert_between(Tp,'Myr','s','t')
        
        NBo, KS, PS = PyUL.NBodyEnergy(MassListSI,TMDataSI,EndNum, a, length_units)
        rP = PyUL.RecPlummer(a,length_units)
        
        # Section 5 Key Information Recreation
        
        mP = PyUL.convert(particles[0][0],m_mass_unit,'m')
        
        mS = PyUL.convert(solitons[0][0],s_mass_unit,'m')
        
        xP = np.array(particles[0][1])
        sP = np.array(solitons[0][1])
        
        xPC = PyUL.convert(xP,m_position_unit,'l')
        sPC = PyUL.convert(sP,s_position_unit,'l')
        
        rC = np.linalg.norm(xPC-sPC)
        
        rU = PyUL.convert_back(rC,'kpc','l')
        
        MaRa = mP / mS
        
        MIn, vRel0 = PyUL.DefaultSolitonOrbit(resol,length, 
                                              length_units, mS, '', rC, '', m_velocity_unit)
        
        MInSI = PyUL.convert_between(MIn,s_mass_unit,'kg','m')
        vRel0SI = PyUL.convert_between(vRel0,m_velocity_unit,'m/s','v')
        MBHSI = PyUL.convert_between(mP,m_mass_unit,'kg','m')
        RSI = PyUL.convert_between(rC,'','m','l')
        
        Density0 = PyUL.ReadLocalDensity(loc)
        DensitySI = PyUL.convert_between(Density0,'','kg/m3','d')
        
        print(MInSI, vRel0SI, MBHSI, RSI, DensitySI)
        
        T = PyUL.convert_between((MInSI)**(3/2) / (4*pi*np.sqrt(PyUL.G)*MBHSI*DensitySI*RSI**(3/2)),'s','Myr','t')

        print(T, 'Myr')
        
        CLow = 1/3 * (PyUL.axion_mass / PyUL.hbar)**2 * PyUL.G*RSI*MInSI
                
        TimeLog.append(Tp)
        ResoLog.append(resol)
        PlRaLog.append(rP)
        
        TMMaLog.append(mP)
        TMDSLog.append(TMDataS[:,0:3])
        TMDVLog.append(TMDataS[:,3:6])
                
        DyTSLog.append(T/CLow)
        IniDLog.append(DensitySI)

        #
        
        IniRLog.append(np.round(rU,4))
        MaRaLog.append(np.round(MaRa,4))
        MEncLog.append(MIn/mS)
        
        
        # Energy
        
        MES = (PS + KS)
        MESD = PyUL.GetRel(MES)

        EKQD = PyUL.GetRel(ekandqlist)
        EGPD = PyUL.GetRel(egpsilist)

        ECMD = PyUL.GetRel(egpcmlist)
        ECOD = PyUL.GetRel(egpcmlist2)

        KSD = PyUL.GetRel(KS)
        PSD = PyUL.GetRel(PS)

        EUOld  = egylist
        EUOldD = PyUL.GetRel(EUOld)

        EUNew  = egpsilist + ekandqlist + egpcmlist2
        EUNewD = PyUL.GetRel(EUNew)

        ETOld  = EUOld + MES
        ETOldD = PyUL.GetRel(ETOld)

        ETNew  = EUNew + MES

        TMEKLog.append(MES)
        TMEPLog.append(egpcmlist2)
        
        ULEKLog.append(ekandqlist)
        ULEPLog.append(egpsilist)
        
        SysELog.append(ETNew)

        TMAcLog.append(AccDataSI)
        
        
        Run += 1
        
        clear_output()
        
        
printU('Processing Complete!','IO')

printU(f'{time.time() - Time:.3f}s','Time')

PyUL2.   IO: Processing Complete!
PyUL2. Time: 9.459s


In [5]:
len(DyTSLog)

45